In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Using rstrip() plus a list comprehension to get a list of all the lines of text reviews.

In [ ]:
df_review = [line.rstrip() for line in open('../input/amazon_alexa.tsv')]
print (len(df_review))

In [ ]:
import pandas
df_review = pandas.read_csv('../input/amazon_alexa.tsv', sep='\t')
df_review.head()

**Basic Exploratory Data Analysis**
Let's check out some of the stats with some plots and the built-in methods in pandas!

In [ ]:
df_review.describe()

In [ ]:
df_review.groupby('rating').describe()

Making a new column to detect how long the text messages are:

In [ ]:
df_review['length'] = df_review['verified_reviews'].apply(len)
df_review.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df_review['length'].plot(bins=50, kind='hist')

Finding out the Maximum length review

In [ ]:
df_review.length.describe()

Wow! 2851 characters long review, let's use masking to find this message:



In [ ]:
df_review[df_review['length'] == 2851]['verified_reviews'].iloc[0]

**Looks like we have some sort of Essay writing review! Now let's focus back on the idea of trying to see if review length is a distinguishing feature between positive and negative review:**

In [ ]:
df_review.hist(column='length', by='feedback', bins=50,figsize=(10,4))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importing the dataset
dataset = pd.read_csv('../input/amazon_alexa.tsv', delimiter = '\t', quoting = 3)


In [ ]:
# Cleaning the texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
for i in range(0,3150):
    review = re.sub('[^a-zA-Z]', ' ', dataset['verified_reviews'][i] )
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

In [ ]:
# creating the Bag of words Model
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,4].values

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


Here I am using XGBoost algorithm to optimize the result

In [ ]:
import xgboost as xgb

In [ ]:
# Fitting Random Forest classifier with 100 trees to the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)


In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

Above shown matrix is known as confusion matrix and below is my result with 
Accuracy of 93.33%
(TP+TN)/total = (576+12)/630 = 0.9333

True Positive Rate: When it's actually yes, how often does it predict yes?
TP/actual yes = 576/576 = 1.00
also known as "Sensitivity" or "Recall"
it is 100%

False Positive Rate: When it's actually no, how often does it predict yes?
FP/actual no = 42/54 = .77
or 77.77%

Precision: When it predicts yes, how often is it correct?
TP/predicted yes = 576/618 = 0.9320 or 93.20%

F-score is 96.48 
